In [1]:
#import sys
import numpy as np
import pandas as pd
%matplotlib inline
#import matplotlib.pyplot as plt
#import seaborn as sns
#import sklearn
import tensorflow as tf
import es_utils as utils

In [2]:
labels = ['label:LYING_DOWN',
 'label:SITTING',
 'label:FIX_walking',
 'label:FIX_running',
 'label:BICYCLING',
 'label:SLEEPING',
 'label:LAB_WORK',
 'label:IN_CLASS',
 'label:IN_A_MEETING',
 'label:LOC_main_workplace',
 'label:OR_indoors',
 'label:OR_outside',
 'label:IN_A_CAR',
 'label:ON_A_BUS',
 'label:DRIVE_-_I_M_THE_DRIVER',
 'label:DRIVE_-_I_M_A_PASSENGER',
 'label:LOC_home',
 'label:FIX_restaurant',
 'label:PHONE_IN_POCKET',
 'label:OR_exercise',
 'label:COOKING',
 'label:SHOPPING',
 'label:STROLLING',
 'label:DRINKING__ALCOHOL_',
 'label:BATHING_-_SHOWER',
 'label:CLEANING',
 'label:DOING_LAUNDRY',
 'label:WASHING_DISHES',
 'label:WATCHING_TV',
 'label:SURFING_THE_INTERNET',
 'label:AT_A_PARTY',
 'label:AT_A_BAR',
 'label:LOC_beach',
 'label:SINGING',
 'label:TALKING',
 'label:COMPUTER_WORK',
 'label:EATING',
 'label:TOILET',
 'label:GROOMING',
 'label:DRESSING',
 'label:AT_THE_GYM',
 'label:STAIRS_-_GOING_UP',
 'label:STAIRS_-_GOING_DOWN',
 'label:ELEVATOR',
 'label:OR_standing',
 'label:AT_SCHOOL',
 'label:PHONE_IN_HAND',
 'label:PHONE_IN_BAG',
 'label:PHONE_ON_TABLE',
 'label:WITH_CO-WORKERS',
 'label:WITH_FRIENDS']

len(labels)

51

In [8]:
# montar os 20 folds da seguinte maneira:
# percorra toda a pasta onde tem os datasets dos usuarios e pegue os `ids`
# selecione 40 aleatoriamente, 20x
# carregue apenas esses 40 datasets (delete-os da memoria ao final) e treine um modelo unificado

In [9]:
k_folds = utils.create_k_folds_n_users(5, 3, '/home/wander/OtherProjects/har_flower/sample_data')

In [11]:

for fold in k_folds:
    config = {'df': k_folds[fold]['train']}
    har = utils.HAR(config)
    har.run()
    
    har.mlp.model.save('model/saved_model')
    converter = tf.lite.TFLiteConverter.from_saved_model('model/saved_model') # path to the SavedModel directory
    tflite_model = converter.convert()
    with open(f'model/model_{fold}.tflite', 'wb') as f:
      f.write(tflite_model)

2022-03-16 16:13:50.886268: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-16 16:13:50.945015: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-03-16 16:13:50.945039: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-03-16 16:13:50.946062: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN

Epoch 1/40
682/682 [==============================] - 1s 930us/step - loss: 0.3797 - categorical_accuracy: 0.7421 - val_loss: 0.3358 - val_categorical_accuracy: 0.7682
Epoch 2/40
682/682 [==============================] - 0s 727us/step - loss: 0.3326 - categorical_accuracy: 0.7738 - val_loss: 0.3239 - val_categorical_accuracy: 0.7705
Epoch 3/40
682/682 [==============================] - 0s 723us/step - loss: 0.3217 - categorical_accuracy: 0.7763 - val_loss: 0.3151 - val_categorical_accuracy: 0.7752
Epoch 4/40
682/682 [==============================] - 1s 747us/step - loss: 0.3157 - categorical_accuracy: 0.7780 - val_loss: 0.3106 - val_categorical_accuracy: 0.7752
Epoch 5/40
682/682 [==============================] - 1s 763us/step - loss: 0.3115 - categorical_accuracy: 0.7795 - val_loss: 0.3088 - val_categorical_accuracy: 0.7776
Epoch 6/40
682/682 [==============================] - 1s 774us/step - loss: 0.3086 - categorical_accuracy: 0.7814 - val_loss: 0.3052 - val_categorical_accuracy:

2022-03-16 16:14:11.997449: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: model/saved_model/assets


2022-03-16 16:14:12.480336: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:363] Ignored output_format.
2022-03-16 16:14:12.480358: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:366] Ignored drop_control_dependency.
2022-03-16 16:14:12.480363: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:372] Ignored change_concat_input_ranges.
2022-03-16 16:14:12.481052: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: model/saved_model
2022-03-16 16:14:12.482418: I tensorflow/cc/saved_model/reader.cc:107] Reading meta graph with tags { serve }
2022-03-16 16:14:12.482433: I tensorflow/cc/saved_model/reader.cc:148] Reading SavedModel debug info (if present) from: model/saved_model
2022-03-16 16:14:12.485599: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2022-03-16 16:14:12.513091: I tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: model/saved_model

...

INFO:tensorflow:Assets written to: model/saved_model/assets
2022-03-16 16:14:49.702533: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:363] Ignored output_format.
2022-03-16 16:14:49.702551: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:366] Ignored drop_control_dependency.
2022-03-16 16:14:49.702556: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:372] Ignored change_concat_input_ranges.
2022-03-16 16:14:49.702731: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: model/saved_model
2022-03-16 16:14:49.703626: I tensorflow/cc/saved_model/reader.cc:107] Reading meta graph with tags { serve }
2022-03-16 16:14:49.703638: I tensorflow/cc/saved_model/reader.cc:148] Reading SavedModel debug info (if present) from: model/saved_model
2022-03-16 16:14:49.706238: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2022-03-16 16:14:49.729252: I tensorflow/cc/saved_model/loader.cc:212] Running initi

Epoch 1/40
809/809 [==============================] - 1s 1ms/step - loss: 0.3994 - categorical_accuracy: 0.6869 - val_loss: 0.3492 - val_categorical_accuracy: 0.7264
Epoch 2/40
809/809 [==============================] - 1s 837us/step - loss: 0.3478 - categorical_accuracy: 0.7098 - val_loss: 0.3330 - val_categorical_accuracy: 0.7308
Epoch 3/40
809/809 [==============================] - 1s 842us/step - loss: 0.3369 - categorical_accuracy: 0.7140 - val_loss: 0.3257 - val_categorical_accuracy: 0.7392
Epoch 4/40
809/809 [==============================] - 1s 818us/step - loss: 0.3310 - categorical_accuracy: 0.7181 - val_loss: 0.3206 - val_categorical_accuracy: 0.7412
Epoch 5/40
809/809 [==============================] - 1s 823us/step - loss: 0.3268 - categorical_accuracy: 0.7223 - val_loss: 0.3174 - val_categorical_accuracy: 0.7442
Epoch 6/40
809/809 [==============================] - 1s 841us/step - loss: 0.3236 - categorical_accuracy: 0.7288 - val_loss: 0.3167 - val_categorical_accuracy: 0

INFO:tensorflow:Assets written to: model/saved_model/assets
2022-03-16 16:15:14.780682: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:363] Ignored output_format.
2022-03-16 16:15:14.780701: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:366] Ignored drop_control_dependency.
2022-03-16 16:15:14.780705: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:372] Ignored change_concat_input_ranges.
2022-03-16 16:15:14.780877: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: model/saved_model
2022-03-16 16:15:14.781759: I tensorflow/cc/saved_model/reader.cc:107] Reading meta graph with tags { serve }
2022-03-16 16:15:14.781772: I tensorflow/cc/saved_model/reader.cc:148] Reading SavedModel debug info (if present) from: model/saved_model
2022-03-16 16:15:14.784259: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2022-03-16 16:15:14.807508: I tensorflow/cc/saved_model/loader.cc:212] Running initi

Epoch 1/40
576/576 [==============================] - 1s 1ms/step - loss: 0.4343 - categorical_accuracy: 0.6364 - val_loss: 0.3835 - val_categorical_accuracy: 0.6611
Epoch 2/40
576/576 [==============================] - 0s 821us/step - loss: 0.3800 - categorical_accuracy: 0.6505 - val_loss: 0.3644 - val_categorical_accuracy: 0.6597
Epoch 3/40
576/576 [==============================] - 0s 806us/step - loss: 0.3677 - categorical_accuracy: 0.6524 - val_loss: 0.3554 - val_categorical_accuracy: 0.6625
Epoch 4/40
576/576 [==============================] - 0s 797us/step - loss: 0.3604 - categorical_accuracy: 0.6564 - val_loss: 0.3510 - val_categorical_accuracy: 0.6681
Epoch 5/40
576/576 [==============================] - 0s 835us/step - loss: 0.3556 - categorical_accuracy: 0.6642 - val_loss: 0.3455 - val_categorical_accuracy: 0.6917
Epoch 6/40
576/576 [==============================] - 0s 794us/step - loss: 0.3518 - categorical_accuracy: 0.7100 - val_loss: 0.3422 - val_categorical_accuracy: 0

INFO:tensorflow:Assets written to: model/saved_model/assets
2022-03-16 16:15:33.489455: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:363] Ignored output_format.
2022-03-16 16:15:33.489476: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:366] Ignored drop_control_dependency.
2022-03-16 16:15:33.489482: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:372] Ignored change_concat_input_ranges.
2022-03-16 16:15:33.489656: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: model/saved_model
2022-03-16 16:15:33.490532: I tensorflow/cc/saved_model/reader.cc:107] Reading meta graph with tags { serve }
2022-03-16 16:15:33.490544: I tensorflow/cc/saved_model/reader.cc:148] Reading SavedModel debug info (if present) from: model/saved_model
2022-03-16 16:15:33.493042: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2022-03-16 16:15:33.516063: I tensorflow/cc/saved_model/loader.cc:212] Running initi

In [12]:
# faca uma otimizacao de parametros por algum tipo de grid ou random search (ou evolutivo, se der tempo)

In [14]:
config = {'df_path': '../input/user1.features_labels.csv'}
har = utils.HAR(config)
har.run()

Epoch 1/40
161/161 [==============================] - 1s 3ms/step - loss: 0.4426 - categorical_accuracy: 0.6530 - val_loss: 0.3884 - val_categorical_accuracy: 0.6915
Epoch 2/40
161/161 [==============================] - 0s 848us/step - loss: 0.3471 - categorical_accuracy: 0.8022 - val_loss: 0.3235 - val_categorical_accuracy: 0.8358
Epoch 3/40
161/161 [==============================] - 0s 897us/step - loss: 0.3121 - categorical_accuracy: 0.8265 - val_loss: 0.3077 - val_categorical_accuracy: 0.8184
Epoch 4/40
161/161 [==============================] - 0s 889us/step - loss: 0.2935 - categorical_accuracy: 0.8277 - val_loss: 0.2915 - val_categorical_accuracy: 0.8358
Epoch 5/40
161/161 [==============================] - 0s 830us/step - loss: 0.2826 - categorical_accuracy: 0.8296 - val_loss: 0.2819 - val_categorical_accuracy: 0.8358
Epoch 6/40
161/161 [==============================] - 0s 871us/step - loss: 0.2751 - categorical_accuracy: 0.8333 - val_loss: 0.2756 - val_categorical_accuracy: 0

In [3]:
config = {'df_path': '../input/user1.features_labels.csv'}
har = utils.HAR(config)
model, best_hps, best_epoch, test_results, ba = har.hypertunning()

Trial 30 Complete [00h 00m 01s]
val_categorical_accuracy: 0.3333333432674408

Best val_categorical_accuracy So Far: 0.8706467747688293
Total elapsed time: 00h 00m 25s
INFO:tensorflow:Oracle triggered exit
Epoch 1/50
51/51 [==============================] - 0s 3ms/step - loss: 0.3547 - categorical_accuracy: 0.7201 - val_loss: 0.2850 - val_categorical_accuracy: 0.7687
Epoch 2/50
51/51 [==============================] - 0s 1ms/step - loss: 0.2398 - categorical_accuracy: 0.8159 - val_loss: 0.2303 - val_categorical_accuracy: 0.8159
Epoch 3/50
51/51 [==============================] - 0s 1ms/step - loss: 0.2141 - categorical_accuracy: 0.8327 - val_loss: 0.2079 - val_categorical_accuracy: 0.8408
Epoch 4/50
51/51 [==============================] - 0s 1ms/step - loss: 0.2007 - categorical_accuracy: 0.8458 - val_loss: 0.2058 - val_categorical_accuracy: 0.8408
Epoch 5/50
51/51 [==============================] - 0s 1ms/step - loss: 0.1958 - categorical_accuracy: 0.8439 - val_loss: 0.2057 - val_cate

51/51 [==============================] - 0s 1ms/step - loss: 0.1509 - categorical_accuracy: 0.8868 - val_loss: 0.1711 - val_categorical_accuracy: 0.8756
Epoch 50/50
51/51 [==============================] - 0s 1ms/step - loss: 0.1509 - categorical_accuracy: 0.8843 - val_loss: 0.1713 - val_categorical_accuracy: 0.8781
Best epoch: 34
Epoch 1/34
161/161 [==============================] - 0s 1ms/step - loss: 0.2979 - categorical_accuracy: 0.7587 - val_loss: 0.2146 - val_categorical_accuracy: 0.8408
Epoch 2/34
161/161 [==============================] - 0s 822us/step - loss: 0.2073 - categorical_accuracy: 0.8364 - val_loss: 0.1980 - val_categorical_accuracy: 0.8582
Epoch 3/34
161/161 [==============================] - 0s 829us/step - loss: 0.1917 - categorical_accuracy: 0.8458 - val_loss: 0.1914 - val_categorical_accuracy: 0.8607
Epoch 4/34
161/161 [==============================] - 0s 804us/step - loss: 0.1821 - categorical_accuracy: 0.8632 - val_loss: 0.1890 - val_categorical_accuracy: 0.87

In [4]:
best_epoch

34

In [5]:
best_hps.values

{'units_1': 60,
 'learning_rate': 0.1,
 'momentum': 0.8,
 'l2': 0.0001,
 'num_layers': 1,
 'units_2': 60,
 'tuner/epochs': 10,
 'tuner/initial_epoch': 4,
 'tuner/bracket': 2,
 'tuner/round': 2,
 'tuner/trial_id': '1baaa5f8977bd0c6e73cb0453740384f'}

In [6]:
ba

0.8389692287849785

In [7]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 60)                13680     
                                                                 
 dense_6 (Dense)             (None, 4)                 244       
                                                                 
Total params: 13,924
Trainable params: 13,924
Non-trainable params: 0
_________________________________________________________________
